# Data setup

In [ ]:
# Paths
try:
  import google.colab
  google.colab.drive.mount('/content/drive')
  root = 'path_to_main_drive_folder/'
except:
  root= 'path_to_main_local_folder/'

data_root = 
mmdet_root = 

# Paths to the COCO-style jsons with all the annotations
anno_train = data_root + 'train.json'
anno_valid = data_root + 'valid.json'
anno_test = data_root + 'test.json'

# Folders of the images
train_folder = data_root + 'images/train'
valid_folder = data_root + 'images/valid'
test_folder = data_root + 'images/test'

# Working directory
work_dir = root + 'models/'

# All the classes in the dataset
classes = ('', '', '')

In [ ]:
# Hyperparameters

h, w = 640, 640

hyperparameters = {
    'MODEL_DESCRIPTION': '',
    'BS': 8,
    'EPOCHS': 30,
    'IMG_SIZE': (h, w),      # (height, width)
    'LR': 5e-5,
    # 'MOMS': (0.85, 0.95),
    'LR_DIVS': (30, 1e4),
    'WARMUP_PCT': 1/3,    
    'TRANSFORMS': [
        dict(type='RandomAffine', max_rotate_degree=5.0, max_translate_ratio=0.0, scaling_ratio_range=(1.0, 1.0), max_shear_degree=1.0),
        dict(type='RandomCrop', crop_size=(0.8,0.8), crop_type='relative_range', allow_negative_crop=True),
        dict(type='Resize', scale=(h, w)),
        dict(type='RandomFlip', direction=['horizontal', 'vertical'], prob=[1/2,1/2]),
        dict(type='PhotoMetricDistortion', brightness_delta=25, contrast_range=(0.7, 1.3), saturation_range=(0.7, 1.3), hue_delta=25),
    ],
    'ARCH': 'dino-4scale_r50',
    'BASE_CONFIG': 'configs/dino/dino-4scale_r50_improved_8xb2-12e_coco.py',
    'WEIGHTS': 'https://download.openmmlab.com/mmdetection/v3.0/dino/dino-4scale_r50_improved_8xb2-12e_coco/dino-4scale_r50_improved_8xb2-12e_coco_20230818_162607-6f47a913.pth',
    'SEED': 18
}

In [ ]:
from mmengine import Config
from mmengine.runner import set_random_seed
from mmengine.runner import Runner

In [ ]:
from DLOlympus.mmdetection.hooks import ValidationLossHook

# Load base config file
cfg = Config.fromfile(mmdet_root+hyperparameters['BASE_CONFIG'])

# Modify dataset classes
cfg.metainfo = {
    'classes': classes,
}

# Modify dataset files
cfg.data_root = data_root

if 'dataset' in cfg.train_dataloader.dataset.keys(): cfg.train_dataloader.dataset = cfg.train_dataloader.dataset.dataset # hotfix
cfg.train_dataloader.dataset.ann_file = anno_train
cfg.train_dataloader.dataset.data_root = data_root
cfg.train_dataloader.dataset.data_prefix.img = train_folder
cfg.train_dataloader.dataset.metainfo = cfg.metainfo
cfg.train_dataloader.dataset.filter_cfg.filter_empty_gt = False

cfg.val_dataloader.dataset.ann_file = anno_valid
cfg.val_dataloader.dataset.data_root = data_root
cfg.val_dataloader.dataset.data_prefix.img = valid_folder
cfg.val_dataloader.dataset.metainfo = cfg.metainfo

cfg.test_dataloader.dataset.ann_file = anno_test
cfg.test_dataloader.dataset.data_root = data_root
cfg.test_dataloader.dataset.data_prefix.img = test_folder
cfg.test_dataloader.dataset.metainfo = cfg.metainfo

# Modify metric config
cfg.val_evaluator.ann_file = anno_valid
cfg.val_evaluator.classwise = True
cfg.test_evaluator.ann_file = anno_test
cfg.test_evaluator.classwise = True

# Modify num classes of each model head
if 'bbox_head' in cfg.model:
    cfg.model.bbox_head.num_classes = len(classes)
if 'segm_head' in cfg.model:
    cfg.model.mask_head.num_classes = len(classes)
if 'roi_head' in cfg.model:
    if 'bbox_head' in cfg.model.roi_head:
        if 'num_classes' in cfg.model.roi_head.bbox_head:
            cfg.model.roi_head.bbox_head.num_classes = len(classes)
        else:
            for head in cfg.model.roi_head.bbox_head:
                head.num_classes = len(classes)
    if 'segm_head' in cfg.model.roi_head:
        if 'num_classes' in cfg.model.roi_head.segm_head:
            cfg.model.roi_head.segm_head.num_classes = len(classes)
        else:
            for head in cfg.model.roi_head.segm_head:
                head.num_classes = len(classes)


if 'roi_head' in cfg.model:
    if 'semantic_roi_extractor' in cfg.model.roi_head:
        cfg.model.roi_head.semantic_roi_extractor = None
    if 'semantic_head' in cfg.model.roi_head:
        cfg.model.roi_head.semantic_head = None


# Put gt boxes in CPU
# for i in cfg.model.train_cfg:
#     if 'assigner' in i:
#         i.assigner.gpu_assign_thr = 0
#         i.assigner.pos_iou_thr = 0

# Use checkpoint to save memory (but slower)
cfg.model.backbone.with_cp = True

# FP16
cfg.fp16 = dict(loss_scale='dynamic')

# Load pretrained weights
cfg.load_from = hyperparameters['WEIGHTS']

# Set up working dir to save files and logs
cfg.work_dir = work_dir

# Save weights for the epoch with best metrics
cfg.default_hooks.checkpoint.interval = -1
cfg.default_hooks.checkpoint.save_best = 'coco/bbox_mAP'
# cfg.default_hooks.checkpoint.save_best = 'coco/segm_mAP'
# Frequency for logging
cfg.default_hooks.logger.interval = 100

cfg.param_scheduler = [
    # LR
    dict(type='CosineAnnealingLR',
        T_max = round(hyperparameters['WARMUP_PCT'] * hyperparameters['EPOCHS']),
        eta_min = hyperparameters['LR'],
        begin = 0,
        end = round(hyperparameters['WARMUP_PCT'] * hyperparameters['EPOCHS']),
        by_epoch=True,
        convert_to_iter_based=True),
    dict(type='CosineAnnealingLR',
        T_max = hyperparameters['EPOCHS'] - round(hyperparameters['WARMUP_PCT'] * hyperparameters['EPOCHS']) - 1,
        eta_min = hyperparameters['LR'] / hyperparameters['LR_DIVS'][1],
        begin = round(hyperparameters['WARMUP_PCT'] * hyperparameters['EPOCHS']),
        end = hyperparameters['EPOCHS'] - 1,
        by_epoch=True,
        convert_to_iter_based=True),
]

# Initial lr and momentum
cfg.optim_wrapper.optimizer.lr = hyperparameters['LR'] / hyperparameters['LR_DIVS'][0]
# cfg.optim_wrapper.optimizer.momentum = hyperparameters['MOMS'][1]
cfg.optim_wrapper.optimizer.type = 'AdamW'
cfg.optim_wrapper.optimizer.weight_decay = 0.05
if 'momentum' in cfg.optim_wrapper.optimizer:
    del cfg.optim_wrapper.optimizer.momentum

# Set seed
cfg.seed = hyperparameters['SEED']
set_random_seed(hyperparameters['SEED'], deterministic=False)

# We can also use tensorboard to log the training process
cfg.visualizer.vis_backends.append({"type":'TensorboardVisBackend'})

# Gradient clipping
cfg.optimizer_config = dict(grad_clip=dict(max_norm=35, norm_type=2))

# Batch size
cfg.train_dataloader.batch_size = hyperparameters['BS']
cfg.val_dataloader.batch_size = hyperparameters['BS']
cfg.test_dataloader.batch_size = hyperparameters['BS']

# Epochs
cfg.train_cfg = dict(max_epochs=hyperparameters['EPOCHS'], type='EpochBasedTrainLoop', val_interval=1)

# Validation loss log
cfg.custom_hooks = [dict(type='ValidationLossHook',
                    loss_name='total_loss',       # Name for logging
                    calculation_method='loss',    # Explicitly use model.loss()
                    force_train_mode=True,        # Explicitly force train()
                    sum_all_components=True,      # Explicitly sum all
                    interval=1)]

# ------------------ DATA AUGMENTATION CONFIG ------------------

# Update train pipeline
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=False, with_seg=False, poly2mask=False),
] + hyperparameters['TRANSFORMS'] + [dict(type='PackDetInputs'),]

# Update valid pipeline
valid_pipeline=[
            dict(type='LoadImageFromFile'),
            dict(type='Resize', keep_ratio=True, scale=hyperparameters['IMG_SIZE']),
            dict(type='LoadAnnotations', with_bbox=True),
            dict(
                meta_keys=(
                    'img_id',
                    'img_path',
                    'ori_shape',
                    'img_shape',
                    'scale_factor',
                ),
                type='PackDetInputs'),
]

# Set all pipelines
cfg.train_dataloader.dataset.pipeline = train_pipeline
cfg.val_dataloader.dataset.pipeline = valid_pipeline
cfg.test_dataloader.dataset.pipeline = valid_pipeline

In [ ]:
from DLOlympus.mmdetection.visualization import show_samples

# Show samples
show_samples(cfg, unique=False)

In [ ]:
# Show transforms
show_samples(cfg, unique=True)

# Training

In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

# Build the runner from config
runner = Runner.from_cfg(cfg)
# Start training
runner.train()

# Results and logs

In [ ]:
import os, shutil

# Get folder where the files are saved
work_subdir = max([work_dir + d for d in os.listdir(work_dir) if os.path.isdir(work_dir + d)], key=os.path.getmtime) + '/'
# Move model and log to the rigth folder
config_path = work_subdir + 'vis_data/' + 'config.py'
model_path = work_dir + [i for i in os.listdir(work_dir) if 'best' in i][0]
os.rename(config_path, work_subdir+'config.py')
os.rename(model_path, work_subdir+[i for i in os.listdir(work_dir) if 'best' in i][0])
config_path = work_subdir+'config.py'
model_path = work_subdir+[i for i in os.listdir(work_subdir) if 'best' in i][0]
# Delete unnecesary files
shutil.rmtree(work_subdir+'vis_data')

In [ ]:
from DLOlympus.mmdetection.plots import plot_losses, plot_metrics
from DLOlympus.mmdetection.utils import get_metrics

log_path = work_subdir + work_subdir.split('/')[-2] + '.log'

_ = plot_losses(log_path, work_subdir)
_ = plot_metrics(log_path, work_subdir)
results = get_metrics(log_path)

# Test

In [ ]:
%cd {mmdet_root}

# Save results
!python tools/test.py \
    {config_path} \
    {model_path} \
    --out {work_subdir+'results.pkl'} \
    --cfg-options custom_hooks="[]"
    # --show \
    # --show-dir {work_subdir+'preds'}

In [ ]:
from DLOlympus.mmdetection.evaluation import collect_detections_and_ground_truths
from mmdet.utils import replace_cfg_vals, update_data_root
from mmdet.registry import DATASETS
from mmengine.registry import init_default_scope
from mmengine.fileio import load

cfg = replace_cfg_vals(cfg)
update_data_root(cfg)
init_default_scope(cfg.get('default_scope', 'mmdet'))
results = load(work_subdir+'results.pkl')
dataset = DATASETS.build(cfg.test_dataloader.dataset)

pred_classes, gt_classes = collect_detections_and_ground_truths(dataset, results, score_thr=0.3, tp_iou_thr=0.5)

In [ ]:
from DLOlympus.utils.plots import plot_confusion_matrix
import itertools

new_vocab = [' '.join(i) for i in list(itertools.product(list(classes)+['background']))]
_ = plot_confusion_matrix(gt_classes, pred_classes, new_vocab, work_subdir+'confusion.png')